In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
print(tf.__version__)

2.4.1


In [4]:
digits = pd.read_csv('../data/digit_recognizer_train.csv')

In [5]:
digits.shape

(42000, 785)

In [6]:
digits.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [7]:
X = digits.iloc[:, 1:]
y = digits.iloc[:, 0]

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [12]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Flatten, Dense, Input, concatenate

In [14]:
input_wide = Input(shape=784, name="wide_input")

input_deep = Input(shape=784, name="deep_input")
hidden1 = Dense(16, activation="relu")(input_deep)
hidden2 = Dense(16, activation="relu")(hidden1)

concat = concatenate([input_wide, hidden2])

output = Dense(10, name="main_output", activation = 'softmax')(concat)

aux_output = Dense(10, name="aux_output", activation = 'softmax')(hidden2)

model = Model(inputs=[input_wide, input_deep],
                           outputs=[output, aux_output])

In [15]:
model.compile(optimizer='sgd', loss ='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [16]:
model.fit([X_train, X_train], [y_train.values, y_train.values], epochs = 10, batch_size= 32)

Epoch 1/10
880/880 [==============================] - 1s 864us/step - loss: 2.2955 - main_output_loss: 0.8695 - aux_output_loss: 1.4260 - main_output_accuracy: 0.7391 - aux_output_accuracy: 0.5522
Epoch 2/10
880/880 [==============================] - 1s 831us/step - loss: 0.7200 - main_output_loss: 0.3302 - aux_output_loss: 0.3898 - main_output_accuracy: 0.9059 - aux_output_accuracy: 0.8859
Epoch 3/10
880/880 [==============================] - 1s 841us/step - loss: 0.5796 - main_output_loss: 0.2781 - aux_output_loss: 0.3015 - main_output_accuracy: 0.9211 - aux_output_accuracy: 0.9121
Epoch 4/10
880/880 [==============================] - 1s 862us/step - loss: 0.5012 - main_output_loss: 0.2457 - aux_output_loss: 0.2555 - main_output_accuracy: 0.9290 - aux_output_accuracy: 0.9242
Epoch 5/10
880/880 [==============================] - 1s 818us/step - loss: 0.4342 - main_output_loss: 0.2147 - aux_output_loss: 0.2196 - main_output_accuracy: 0.9369 - aux_output_accuracy: 0.9356
Epoch 6/10
880/

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 16)           12560       deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 16)           272         dense[0][0]                      
______________________________________________________________________________________________

In [18]:
model.evaluate([X_train, X_train], [y_train, y_train])   #Output:[Loss, accuracy_score]

880/880 [==============================] - 1s 657us/step - loss: 0.2705 - main_output_loss: 0.1345 - aux_output_loss: 0.1360 - main_output_accuracy: 0.9604 - aux_output_accuracy: 0.9603


[0.2705462872982025,
 0.13450349867343903,
 0.13604296743869781,
 0.9604477882385254,
 0.9603056311607361]

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
output, aux_output = model.predict([X_test, X_test])

In [21]:
np.argmax(output, axis=1)

array([8, 1, 9, ..., 6, 3, 2])

In [22]:
accuracy_score(y_test, np.argmax(output, axis=1))

0.9378066378066378

In [23]:
accuracy_score(y_test, np.argmax(aux_output, axis=1))

0.9341991341991343